In [13]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F

In [14]:
# 加载normal.npy和abnormal.npy文件中的数据  
normal_data = np.load('./voice/data/normal.npy')  
abnormal_data = np.load('./voice/data/abnormal.npy')  


# 合并正常和异常数据
data = np.concatenate((normal_data, abnormal_data), axis=0)

# 创建标签，0 表示正常，1 表示异常
labels = np.concatenate((np.zeros(len(normal_data)), np.ones(len(abnormal_data))), axis=0)

# 划分训练集和测试集，测试集占比为 20%，随机种子为 42
train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=42
)
train_labels_tensor = torch.tensor(train_labels, dtype=torch.float32)
test_labels_tensor = torch.tensor(test_labels, dtype=torch.float32)



# 数据标准化
scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))
test_data_normalized = scaler.transform(test_data.reshape(-1, 1))
# 转换为 PyTorch 张量
train_data_tensor = torch.FloatTensor(train_data_normalized).view(-1)
test_data_tensor = torch.FloatTensor(test_data_normalized).view(-1)



In [15]:
# 定义 LSTM 模型
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input_seq):
        lstm_out, _ = self.lstm(input_seq.view(len(input_seq), 1, -1))
        predictions = self.linear(lstm_out.view(len(input_seq), -1))
        return predictions[-1]

input_size = 1
hidden_size = 64
output_size = 1
model = LSTMModel(input_size, hidden_size, output_size)
train = model(train_data_tensor)


In [16]:

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [17]:

epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(train_data_tensor)
    loss = criterion(output, train_labels_tensor)
    loss.backward()
    optimizer.step()
    if (epoch + 1) % 1 == 0:
        print(f"Epoch [{epoch + 1}/{epochs}], Loss: {loss.item():.4f}")

print(output.size())
print(f"训练完成，最终损失：{loss.item():.4f}")

/opt/conda/envs/work/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([80])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [1/10], Loss: 0.5718
Epoch [2/10], Loss: 0.5506
Epoch [3/10], Loss: 0.5303
Epoch [4/10], Loss: 0.5107
Epoch [5/10], Loss: 0.4918
Epoch [6/10], Loss: 0.4734
Epoch [7/10], Loss: 0.4554
Epoch [8/10], Loss: 0.4377
Epoch [9/10], Loss: 0.4203
Epoch [10/10], Loss: 0.4031
torch.Size([1])
训练完成，最终损失：0.4031


In [18]:
model.eval()
test_loss = 0.0
epochs = 10
for epoch in range(epochs):
    with torch.no_grad():
        test_predictions = model(test_data_tensor)
        test_loss = criterion(test_predictions, test_labels_tensor)
        
        # 将二进制预测转换为整数（0或1）
        binary_predictions = (test_predictions >= 0.5).int()
        
        # 计算准确率
        correct_predictions = (binary_predictions == test_labels_tensor).sum().item()
        total_samples = len(test_labels_tensor)
        accuracy = correct_predictions / total_samples
        
        print(f"Epoch [{epoch + 1}/{epochs}], 测试损失：{test_loss.item():.4f}, 测试准确率：{accuracy:.4f}")
print(test_labels_tensor.size())
print(binary_predictions.size())

Epoch [1/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [2/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [3/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [4/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [5/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [6/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [7/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [8/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [9/10], 测试损失：0.3000, 测试准确率：0.6000
Epoch [10/10], 测试损失：0.3000, 测试准确率：0.6000
torch.Size([20])
torch.Size([1])


/opt/conda/envs/work/lib/python3.11/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([20])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
